In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn import datasets
import pandas as pd


from math import sin, cos, sqrt, atan2, radians

#sentiment packages
from textblob import TextBlob

In [2]:
# load businesses with national neighbor data
business_df = pd.read_json('completed_distance_df.json', lines=False)


In [156]:
business_df.categories[22]

'Restaurants, Asian Fusion, Sushi Bars'

In [220]:
business_df['is_restaurant'] = business_df[['categories']].applymap(lambda x: helper(x, 'restaurant'))
business_df['is_shopping'] = business_df[['categories']].applymap(lambda x: helper(x, 'shopping'))

In [221]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,hours,chain,tip_count,mean_tip_sentiment,.1_km,.3_km,.1_count,.3_count,is_restaurant,is_shopping
0,tsXCDIijxbgsh980VgRc9g,Federal Companies,401 W Kenyon Rd,Champaign,IL,61820,40.133919,-88.248628,3.0,11,...,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",False,0,0.252695,"[[bd_qQhLpRvcd1rL8XR5Sag, 0.0792927127], [3QMq...","[[bd_qQhLpRvcd1rL8XR5Sag, 0.0792927127], [gnGM...",2,5,0,0
1,fsklFcY47qJIr0mjgobuUg,Ford City,300 Carriage Center Ct,Champaign,IL,61820,40.090444,-88.247430,2.5,3,...,"{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ...",False,0,0.252695,"[[6mYtRm6smtgt29hLxt1vDA, 0.047386416], [eJtmO...","[[YG3cf30rN3K9H56-JOG6Iw, 0.1704667473], [yDfO...",8,30,0,0
2,mofOjB6flg-eAWOFbOkHfQ,ChinaTown Buffet,713 W Marketview Dr,Champaign,IL,61822,40.137270,-88.256043,2.5,72,...,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",False,4,0.495312,"[[8ed4YmEWPr5nfau82-OtCA, 0.0919277205], [AUoY...","[[_9lnbDRvashBE9ey5gAJfg, 0.1604202278], [afvF...",5,23,1,0
3,P4HqDYI1icascvcwca7iLg,Macy's,2000 N Neil St,Champaign,IL,61820,40.141029,-88.244222,2.5,12,...,"{'Monday': '10:0-20:0', 'Tuesday': '9:0-22:0',...",True,2,-0.585000,"[[TQg3tu8U3x8OaC2R4KKh7g, 0.0362019815]]","[[B_dwXxYZvEL032j9kUHz9w, 0.2343176697], [y2V4...",1,50,0,1
4,eezVjNlzIZrXs9GM5O8b2w,Vape Vault,723 S Neil St,Champaign,IL,61820,40.107126,-88.243912,4.0,5,...,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-21:0', '...",False,1,0.494444,"[[N5Kcf0cdnL9GYLSYZrnTVw, 0.07163558140000001]...","[[N5Kcf0cdnL9GYLSYZrnTVw, 0.07163558140000001]...",15,30,0,1


In [219]:
restaurant_words = ['restaurant', 'bar', 'diner', 'pub', 'food']
shopping_words = ['shopping']
def helper(x, business_type):
    if business_type == 'restaurant':
        dictionary = restaurant_words
    elif business_type == 'shopping':
        dictionary = shopping_words
        
    if type(x) != str:
        return 0
    x = x.split(",")
    for word in x:
        for word2 in dictionary:
            if word2.lower() in word.lower():
                return 1
    return 0

In [222]:
#even split of set
from sklearn.model_selection import train_test_split

closed_businesses = business_df[business_df['is_open'] == 0]
open_businesses = business_df[business_df['is_open'] == 1]

random_open, _ = train_test_split(open_businesses, train_size = closed_businesses.shape[0])

even_split_data = pd.concat([closed_businesses, random_open], ignore_index=True)


In [223]:
even_split_features = even_split_data[['stars', 'review_count', 'tip_count', 'chain', '.1_count', '.3_count', 'mean_tip_sentiment', 'is_restaurant', 'is_shopping']]
even_split_targets = even_split_data['is_open']

In [104]:
from sklearn.preprocessing import MinMaxScaler

In [224]:
scaler = MinMaxScaler()
scaler.fit(even_split_features)
normalized_features = scaler.transform(even_split_features)

even_split_targets = even_split_targets.to_numpy()

In [226]:
#do not touch
training_features, test_features, training_targets, test_targets = train_test_split(normalized_features, even_split_targets, test_size = 1)


In [ ]:
#experiment zone
experiment_even_split_features = even_split_data[['stars', 'review_count', 'tip_count', 'chain', '.1_count', '.3_count', 'mean_tip_sentiment']]
experiment_even_split_targets = even_split_data['is_open']

experiment_scaler = MinMaxScaler()
experiment_scaler.fit(even_split_features)
experiment_normalized_features = scaler.transform(even_split_features)

experiment_even_split_targets = even_split_targets.to_numpy()

experiment_training_features, experiment_test_features, experiment_training_targets, experiment_test_targets = train_test_split(experiment_normalized_features, experiment_even_split_targets, test_size = 1)


In [227]:
clf = LogisticRegression()
scores = cross_val_score(clf, training_features, training_targets, cv=5)
print(scores)

[0.65836066 0.65486339 0.6504918  0.64721311 0.63300546]


In [228]:
clf = ensemble.GradientBoostingClassifier()
scores = cross_val_score(clf, training_features, training_targets, cv=5)
print('Illinois Businesses Boosting Classifier Scores', scores)
print('Illinois Businesses Boosting Classifier Mean Score', np.mean(scores))

Illinois Businesses Boosting Classifier Scores [0.68546448 0.66863388 0.68393443 0.67060109 0.66666667]
Illinois Businesses Boosting Classifier Mean Score 0.6750601092896175


In [137]:
clf = LogisticRegression()
scores = cross_val_score(clf, training_features, training_targets, cv=5, scoring='roc_auc')
print('National Neighbor Data Logistic Regression ROC AUC scores', scores)
print('National Neighbor Data Logistic Regression ROC AUC Mean score', np.mean(scores))
print()

clf = ensemble.GradientBoostingClassifier()
scores = cross_val_score(clf, training_features, training_targets, cv=5, scoring='roc_auc')
print('National Neighbor Data Boosting Classifier ROC AUC scores', scores)
print('National Neighbor Data Boosting Classifier ROC AUC Mean score', np.mean(scores))

National Neighbor Data Logistic Regression ROC AUC scores [0.6490527  0.63211656 0.63835345 0.64230937 0.63410408]
National Neighbor Data Logistic Regression ROC AUC Mean score 0.6391872311116955

National Neighbor Data Boosting Classifier ROC AUC scores [0.70735397 0.69580649 0.68652956 0.69930529 0.68421973]
National Neighbor Data Boosting Classifier ROC AUC Mean score 0.694643007298779


In [126]:
clf = ensemble.GradientBoostingClassifier()
scores = cross_val_score(clf, training_features, training_targets, cv=5)
print('Illinois Businesses Boosting Classifier Scores', scores)
print('Illinois Businesses Boosting Classifier Mean Score', np.mean(scores))

Illinois Businesses Boosting Classifier Scores [0.63825137 0.64546448 0.63300546 0.63693989 0.62622951]
Illinois Businesses Boosting Classifier Mean Score 0.6359781420765028
